In [1]:
import warnings

from limix._data import conform_dataset, normalize_likelihood
from limix._display import session_block

class VarDec(object):
    """
    Variance decompositon through GLMMs.

    Example
    -------

    .. doctest::

        >>> from limix.vardec import VarDec
        >>> from limix.stats import multivariate_normal as mvn
        >>> from numpy import ones, eye, concatenate, zeros, exp
        >>> from numpy.random import RandomState
        >>>
        >>> random = RandomState(0)
        >>> nsamples = 20
        >>>
        >>> M = random.randn(nsamples, 2)
        >>> M = (M - M.mean(0)) / M.std(0)
        >>> M = concatenate((ones((nsamples, 1)), M), axis=1)
        >>>
        >>> K0 = random.randn(nsamples, 10)
        >>> K0 = K0 @ K0.T
        >>> K0 /= K0.diagonal().mean()
        >>> K0 += eye(nsamples) * 1e-4
        >>>
        >>> K1 = random.randn(nsamples, 10)
        >>> K1 = K1 @ K1.T
        >>> K1 /= K1.diagonal().mean()
        >>> K1 += eye(nsamples) * 1e-4
        >>>
        >>> y = M @ random.randn(3) + mvn(random, zeros(nsamples), K0)
        >>> y += mvn(random, zeros(nsamples), K1)
        >>>
        >>> vardec = VarDec(y, "normal", M)
        >>> vardec.append(K0)
        >>> vardec.append(K1)
        >>> vardec.append_iid()
        >>>
        >>> vardec.fit(verbose=False)
        >>> print(vardec) # doctest: +FLOAT_CMP
        Variance decomposition
        ======================
        <BLANKLINE>
        𝐲 ~ 𝓝(𝙼𝜶, 0.385⋅𝙺 + 1.184⋅𝙺 + 0.000⋅𝙸)
        >>> y = exp((y - y.mean()) / y.std())
        >>> vardec = VarDec(y, "poisson", M)
        >>> vardec.append(K0)
        >>> vardec.append(K1)
        >>> vardec.append_iid()
        >>>
        >>> vardec.fit(verbose=False)
        >>> print(vardec) # doctest: +FLOAT_CMP
        Variance decomposition
        ======================
        <BLANKLINE>
        𝐳 ~ 𝓝(𝙼𝜶, 0.000⋅𝙺 + 0.350⋅𝙺 + 0.000⋅𝙸) for yᵢ ~ Poisson(λᵢ=g(zᵢ)) and g(x)=eˣ
    """
    def __init__(self, y, lik="normal", M=None):
        """
        Constructor.

        Parameters
        ----------
        y : array_like
            Phenotype.
        lik : tuple, "normal", "bernoulli", "probit", "binomial", "poisson"
            Sample likelihood describing the residual distribution.
            Either a tuple or a string specifying the likelihood is required. The
            Normal, Bernoulli, Probit, and Poisson likelihoods can be selected by
            providing a string. Binomial likelihood on the other hand requires a tuple
            because of the number of trials: ``("binomial", array_like)``. Defaults to
            ``"normal"``.
        M : n×c array_like
            Covariates matrix.
        """
        from numpy import asarray
        from glimix_core.mean import LinearMean

        y = asarray(y, float)
        data = conform_dataset(y, M)
        y = data["y"]
        M = data["M"]
        self._y = y
        self._M = M
        self._lik = normalize_likelihood(lik)
        self._mean = LinearMean(asarray(M, float))
        self._covariance = []
        self._glmm = None
        self._fit = False
        self._unnamed = 0


    @property
    def effsizes(self):
        """
        Covariace effect sizes.

        Returns
        -------
        effsizes : ndarray
            Effect sizes.
        """
        if not self._fit:
            self.fit()
        return self._mean.effsizes

    @property
    def covariance(self):
        """
        Get the covariance matrices.

        Returns
        -------
        covariances : list
            Covariance matrices.
        """
        return self._covariance
    def fit(self, verbose=True):
        """
        Fit the model.

        Parameters
        ----------
        verbose : bool, optional
            Set ``False`` to silence it. Defaults to ``True``.
        """
        with session_block("Variance decomposition", disable=not verbose):
            if self._lik[0] == "normal":
                if self._simple_model():
                    self._fit_lmm_simple_model(verbose)
                else:
                    self._fit_lmm(verbose)
            else:
                if self._simple_model():
                    self._fit_glmm_simple_model(verbose)
                else:
                    self._fit_glmm(verbose)

            if verbose:
                print(self)

        self._fit = True
    
    def lml(self):
        """
        Get the log of the marginal likelihood.

        Returns
        -------
        float
            Log of the marginal likelihood.
        """
        if not self._fit:
            self._glmm.fit()
        return self._glmm.lml()
    
    def append_iid(self, name="residual"):
        from glimix_core.cov import EyeCov

        c = EyeCov(self._y.shape[0])
        c.name = name
        self._covariance.append(c)
        
    def append(self, K, name=None):
        from numpy_sugar import is_all_finite
        from numpy import asarray
        from glimix_core.cov import GivenCov

        data = conform_dataset(self._y.values, K=K)
        K = asarray(data["K"], float)

        if not is_all_finite(K):
            raise ValueError("Covariance-matrix values must be finite.")

        K = K / K.diagonal().mean()
        cov = GivenCov(K)
        if name is None:
            name = "unnamed-{}".format(self._unnamed)
            self._unnamed += 1
        cov.name = name

        self._covariance.append(cov)

    def plot(self):
        import limix
        import seaborn as sns
        from matplotlib.ticker import FormatStrFormatter

        variances = [c.scale for c in self._covariance]
        variances = [(v / sum(variances)) * 100 for v in variances]
        names = [c.name for c in self._covariance]

        ax = sns.barplot(x=names, y=variances)
        ax.yaxis.set_major_formatter(FormatStrFormatter("%.0f%%"))
        ax.set_xlabel("random effects")
        ax.set_ylabel("explained variance")
        ax.set_title("Variance decomposition")
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            limix.plot.get_pyplot().tight_layout()
        limix.plot.show()

    def _fit_lmm(self, verbose):
        from glimix_core.cov import SumCov
        from glimix_core.gp import GP
        from numpy import asarray

        y = asarray(self._y, float).ravel()
        gp = GP(y, self._mean, SumCov(self._covariance))
        gp.fit(verbose=verbose)
        self._glmm = gp

    def _fit_glmm(self, verbose):
        from glimix_core.cov import SumCov
        from glimix_core.ggp import ExpFamGP
        from numpy import asarray

        y = asarray(self._y, float).ravel()
        gp = ExpFamGP(y, self._lik, self._mean, SumCov(self._covariance))
        gp.fit(verbose=verbose)
        self._glmm = gp

    def _fit_lmm_simple_model(self, verbose):
        from numpy_sugar.linalg import economic_qs
        from glimix_core.lmm import LMM
        from numpy import asarray

        K = self._get_matrix_simple_model()

        y = asarray(self._y, float).ravel()
        QS = None
        if K is not None:
            QS = economic_qs(K)
        lmm = LMM(y, self._M, QS)
        lmm.fit(verbose=verbose)
        self._set_simple_model_variances(lmm.v0, lmm.v1)
        self._glmm = lmm

    def _fit_glmm_simple_model(self, verbose):
        from numpy_sugar.linalg import economic_qs
        from glimix_core.glmm import GLMMExpFam
        from numpy import asarray

        K = self._get_matrix_simple_model()

        y = asarray(self._y, float).ravel()
        QS = None
        if K is not None:
            QS = economic_qs(K)

        glmm = GLMMExpFam(y, self._lik, self._M, QS)
        glmm.fit(verbose=verbose)

        self._set_simple_model_variances(glmm.v0, glmm.v1)
        self._glmm = glmm

    def _set_simple_model_variances(self, v0, v1):
        from glimix_core.cov import GivenCov, EyeCov

        for c in self._covariance:
            if isinstance(c, GivenCov):
                c.scale = v0
            elif isinstance(c, EyeCov):
                c.scale = v1

    def _get_matrix_simple_model(self):
        from glimix_core.cov import GivenCov

        K = None
        for i in range(len(self._covariance)):
            if isinstance(self._covariance[i], GivenCov):
                self._covariance[i].scale = 1.0
                K = self._covariance[i].value()
                break
        return K

    def _simple_model(self):
        from glimix_core.cov import GivenCov, EyeCov

        if len(self._covariance) > 2:
            return False

        c = self._covariance
        if len(c) == 1 and isinstance(c[0], EyeCov):
            return True

        if isinstance(c[0], GivenCov) and isinstance(c[1], EyeCov):
            return True

        if isinstance(c[1], GivenCov) and isinstance(c[0], EyeCov):
            return True

        return False

    def __repr__(self):
        from glimix_core.cov import GivenCov
        from limix.qtl._result._draw import draw_model
        from limix._display._draw import draw_title
        covariance = ""
        for c in self._covariance:
            s = c.scale
            if isinstance(c, GivenCov):
                covariance += f"{s:.3f}⋅𝙺 + "
            else:
                covariance += f"{s:.3f}⋅𝙸 + "
        if len(covariance) > 2:
            covariance = covariance[:-3]

        msg = draw_title("Variance decomposition")
        msg += draw_model(self._lik[0], "𝙼𝜶", covariance)
        msg = msg.rstrip()

        return msg
    



In [2]:
import limix
from numpy import dot
url = "http://rest.s3for.me/limix/smith08.hdf5.bz2"
filepath = limix.sh.download(url, verbose=False)
filepath = limix.sh.extract(filepath, verbose=False)
data = limix.io.hdf5.read_limix(filepath)
Y = data['phenotype']
G_all = data['genotype']

In [3]:
print(Y)
print(G_all)
lysine_group = [
"YIL094C",
"YDL182W",
"YDL131W",
"YER052C",
"YBR115C",
"YDR158W",
"YNR050C",
"YJR139C",
"YIR034C",
"YGL202W",
"YDR234W"]

<xarray.DataArray 'phenotype' (sample: 109, outcome: 10986)>
array([[-0.03733945, -0.07816514,  0.04293578, ...,  0.09559633,
        -0.13238532, -0.27495413],
       [-0.30137615,  0.06605505,  0.33862385, ..., -0.14266055,
        -0.23834862,  0.73275229],
       [ 0.00266055,  0.12183486, -0.13706422, ..., -0.14440367,
         0.25761468,  0.01504587],
       ...,
       [-0.28733945,  0.35183486,  0.07293578, ...,  0.09733945,
        -0.03834862,  0.16275229],
       [-0.57733945,  0.01183486, -0.00706422, ...,  0.13559633,
         0.10761468,  0.24504587],
       [-0.27733945,  0.06183486,  0.13293578, ...,  0.01559633,
        -0.14238532, -0.12495413]])
Coordinates:
  * sample        (sample) int64 0 1 2 3 4 5 6 7 ... 102 103 104 105 106 107 108
    environment   (outcome) float64 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0
    gene_ID       (outcome) object 'YOL161C' 'YJR107W' ... 'YLR118C' 'YBR242W'
    gene_chrom    (outcome) object '15' '10' '16' '7' '4' ... '3' '10' '1

In [6]:
K_all = dot(G_all, G_all.T)
window_size = int(5e5)
vardecs = []
for gene in lysine_group[:2]:
    # Select the row corresponding to gene of interest on environment 0.0.
    y = Y[:, (Y["gene_ID"] == gene) & (Y["environment"] == 0.0)]
    # Estimated middle point of the gene.
    midpoint = (y["gene_end"].item() - y["gene_start"].item()) / 2
    # Window definition.
    start = midpoint - window_size // 2
    end = midpoint + window_size // 2
    geno = G_all[:, (G_all["pos"] >= start) & (G_all["pos"] <= end)]
    G_cis = G_all[:, geno.candidate]
    K_cis = dot(G_cis, G_cis.T)
    K_trans = K_all - K_cis
    # Definition of the model to fit our data from which we extract
    # the relative signal strength.
    print(y)
    print(len(K_cis))
    print(len(K_trans))
    vardec = VarDec(y, "normal")
    vardec.append(K_cis, "cis")
    vardec.append(K_trans, "trans")
    vardec.append_iid("noise")
    vardec.fit(verbose=False)
    vardecs.append(vardec)

<xarray.DataArray 'phenotype' (sample: 109, outcome: 1)>
array([[ 0.06422018],
       [ 0.33036697],
       [-0.30577982],
       [ 0.20422018],
       [ 0.41036697],
       [-0.04577982],
       [ 0.26036697],
       [ 0.07036697],
       [ 0.45036697],
       [ 0.37422018],
       [ 0.76036697],
       [ 0.15422018],
       [ 0.45422018],
       [ 0.19422018],
       [ 0.68036697],
       [ 0.13422018],
       [ 0.35422018],
       [ 0.48422018],
       [ 0.18036697],
       [ 0.02036697],
       [ 0.02422018],
       [ 0.19422018],
       [ 0.16422018],
       [ 0.45036697],
       [ 0.44422018],
       [-0.08963303],
       [ 0.44036697],
       [ 0.06036697],
       [ 0.01036697],
       [ 0.62036697],
       [-0.18963303],
       [ 0.17036697],
       [-0.04577982],
       [-0.02963303],
       [ 0.25036697],
       [-0.17577982],
       [-0.08963303],
       [-0.25577982],
       [ 0.35036697],
       [ 0.53036697],
       [ 0.17422018],
       [ 0.48422018],
       [ 0.27422018

In [ ]:
print(vardecs[0])
print(vardecs[1])

In [ ]:
vardecs[0].plot()

In [ ]:
vardecs[1].plot()